In [9]:
import os
from openai import OpenAI
from pinecone import Pinecone
from typing import List
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
pinecone_client = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

openai_client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  
)

In [11]:
def get_embedding(text: str, model: str = "text-embedding-3-small") -> List[float]:
    """
    Get the embedding of a text using OpenAI's API.
    """
    response = openai_client.embeddings.create(
        input=text,
        model=model
    )
    embedding = response.data[0].embedding
    return embedding

In [12]:
query = """
Job role is Energy Compliance Consultant, 
Department is Compliance, 
Industry is Energy and Gas.

Functional/Technical Skills:
▪ Effective communicator in both written and verbal communictions, including
ability to propose practical actions and tailor communications to a variety of
levels of stakeholder (including senior management and the Board).
▪ Confident in absorbing and manipulating information/data, from multiple
sources (and multiple formats, e.g Microsoft Excel, PowerBI, operational
systems), to drive Assurance insight, review conclusions and proportionate
actions.
▪ Able to understand, identify and clearly articulate risks and controls arising in
relation to a wide range of issues, with a particular focus on conduct risk and
customer outcomes, and ability to design testing activities in line with the
department methodology.
▪ Robust and independent mindset with the confidence to raise issues
appropriately.
▪ Advanced level stakeholder skills and comfortable dealing with all grades of
colleague.
▪ Able to work effectively individually and as part of a team to deliver to tight
deadlines.

Education and Certifications:
 ▪ Demonstrable experience of compliance activity within a regulatory sector
and/or educated to degree standard (Law or Economics degree preferable)
with aspiration to work in a high performing compliance function.

Competencies:
▪ Demonstrable experience of working in a regulated environment and
understanding of the energy supply regulatory framework.
▪ Credibility to build relationships with a variety of internal stakeholders and
influence accordingly.
▪ Experience in multi-tasking, being able to navigate complex tasks & prioritise
effectively.
▪ Developing close ‘trusted adviser’ relationships with key business leaders.
▪ Wins hearts and minds across the business and inspires others to act for the
betterment of our business and its customers.
▪ Proven oral and written communication skills with the ability to explain
complex compliance requirements and prove pragmatic advice to stakeholders.
▪ Engage with stakeholders in an open, honest, and productive manner to
promote collaborative working.
▪ Effectively manage stakeholders and senior management during meetings by
adapting the style of communication to suit the situation and the individual."""

In [14]:
def fetch_nos_text(query: str) -> List[str]:
    """
    Fetch NOS text from Pinecone based on current role.
    
    """
    query_vector = get_embedding(query)  

    # Query the Pinecone index
    index = pinecone_client.Index('test-nos')  
    nos_docs = index.query(
            vector=query_vector,
            top_k=100,
            include_metadata=True,
            # filter={"industry": "Compliance"}
            # filter={"type":"Developed by"}
            filter={"type":"Performance criteria"}
            # filter={"type":"Knowledge and understanding"}
        )
    return nos_docs['matches']

In [15]:
# nos_docs = fetch_nos_text("Current Job role is Energy Compliance Consultant, Department is Compliance, Industry is Energy and Gas.")['matches']
nos_docs= fetch_nos_text(query)

In [16]:
[(n+1,nos['metadata']['nos_id']) for n,nos in enumerate(nos_docs) if nos['metadata']['nos_id'].startswith('FSPCOMP')]

[(3, 'FSPCOMP7'), (11, 'FSPCOMP12'), (38, 'FSPCOMP9'), (65, 'FSPCOMP5')]

In [18]:
[(nos['metadata']['industry'],nos['metadata']['nos_id'],nos['metadata']['text']) for nos in nos_docs][:11]

[('Anti-money Laundering',
  'FSPAML2',
  "Performance criteria You must be able to: 1. carry out your role in line with your organisation's strategy for compliance 2. communicate agreed values and assumptions that encourage behaviour which is consistent with your organisation’s overall vision and strategy for compliance 3. put in place policies, programmes and systems to support agreed values 4. challenge behaviours and attitudes that conflict with agreed values 5. confirm that senior management recognise, understand and will promote the importance of making compliance part of the culture of your organisation 6. promote the benefits of a positive compliance culture 7. assist in developing a climate of openness about meeting or not meeting the requirements for compliance 8. encourage an ‘open-door’ policy for staff to seek clarification on compliance matters  Assist in developing the compliance culture of your organisation  Assist in developing the compliance culture of your organisati

## To Format NOS text

#### > Generate Skills and Competencies
#### > Include Industry in text (to make it more relevant)